# Generate MetaData for Species Audio Files

This program is designed to create metadata for audio files in a specified directory. It accomplishes this in several steps:

CalcAudioDuration(length): This function calculates the audio duration in seconds, given the length of the audio in milliseconds. It converts milliseconds to seconds and returns the duration.

ComputeAudioDuration(filename): This function retrieves the duration of audio files. It first checks if the file exists and has a supported format (e.g., '.wav' or '.mp3'). If supported, it loads the audio file, calculates its length, and uses CalcAudioDuration to get the duration in seconds. Any errors during this process are caught and printed.

ExtractClassAndFile(path): This function extracts class names and filenames from subdirectories within a specified directory. It lists the subdirectories, iterates through them, and collects the class and filename information.

GetAudioDurations(path, classes, filenames): This function calculates the durations of audio files in the specified directory. It uses the ComputeAudioDuration function to process each audio file, collecting the durations into a list.

CreateMetaData(path): This function generates metadata for the audio files. It combines the class, filename, and duration information into a Pandas DataFrame and returns it.

WriteMetadata(path): This function writes the metadata DataFrame to a CSV file named 'species_metadata.csv' in the specified directory.

The script sets the 'path' variable to the directory containing the audio files and then calls 'WriteMetadata' to generate and save the metadata as a CSV file. This code ensures that the audio file durations are calculated accurately and included in the metadata for further analysis or use.

In [78]:
# Import the libraries
import os
from pydub import AudioSegment
import pandas as pd

In [79]:
def CalcAudioDuration(length):
    """
    Function to compute the duration. You could add more features like hours and minutes here, 
    but for now it would just be seconds.
    """
    seconds = length / 1000  # Convert milliseconds to seconds
    return seconds

def ComputeAudioDuration(filename):
    """
    This function will retrieve the duration from any file passed to it
    """
    try:
        ext = os.path.splitext(filename)[-1].lower()
        if ext not in ['.wav', '.mp3']:
            raise ValueError("Unsupported audio format")

        audio = AudioSegment.from_file(filename)
        length = len(audio)
        seconds = CalcAudioDuration(length)
        return seconds
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return None

def ExtractClassAndFile(path):
    classes = []
    filenames = []
    for folder in os.listdir(path):
        folderpath = os.path.join(path, folder)
        if os.path.isdir(folderpath):
            for file in os.listdir(folderpath):
                if file.endswith(('.wav', '.mp3')):
                    classes.append(folder)
                    filenames.append(file)
    return classes, filenames

def GetAudioDurations(path, classes, filenames):
    seconds = []
    for cls, filename in zip(classes, filenames):
        filepath = os.path.join(path, cls, filename)
        second = ComputeAudioDuration(filepath)
        seconds.append(second)
    return seconds

def CreateMetaData(path):
    classes, filenames = ExtractClassAndFile(path)
    seconds = GetAudioDurations(path, classes, filenames)
    
    feature_colname = ['filename', 'seconds', 'class']
    df = pd.DataFrame({'Filename': filenames, 'Seconds': seconds, 'Class': classes})
    df['Seconds'] = df['Seconds'].fillna(0)
    return df

def WriteMetadata(path):
    df = CreateMetaData(path)
    df.to_csv(os.path.join(path, 'species_metadata.csv'), index=False)

path = "/Users/rahulprem/Uni/SIT782/TestSet"
WriteMetadata(path)

/Users/rahulprem/anaconda3/envs/ProjectEcho/lib/python3.11/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
/Users/rahulprem/anaconda3/envs/ProjectEcho/lib/python3.11/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
/Users/rahulprem/anaconda3/envs/ProjectEcho/lib/python3.11/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


Error loading /Users/rahulprem/Uni/SIT782/TestSet/samples 2/car horn 7.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error loading /Users/rahulprem/Uni/SIT782/TestSet/samples 2/car horn 5.wav: [Errno 2] No such file or directory: 'ffprobe'
Error loading /Users/rahulprem/Uni/SIT782/TestSet/samples 2/car horn 6.wav: [Errno 2] No such file or directory: 'ffprobe'


Finally, the script sets the 'path' variable to "./Data2/" and calls the 'WriteMetadata' function with this path, which generates metadata for audio files in the "./Data2/" directory and saves it in a CSV file named 'species_metadata.csv'.

In [80]:
pd.read_csv('/Users/rahulprem/Uni/SIT782/TestSet/species_metadata.csv')

,Filename,Seconds,Class
0,car horn 7.mp3,0.000,samples 2
1,car horn 4.wav,5.902,samples 2
2,car horn 5.wav,0.000,samples 2
3,car horn 6.wav,0.000,samples 2
4,car horn 8.wav,26.840,samples 3
5,car horn 9.wav,5.611,samples 3
6,car horn 10.wav,0.409,samples 3
7,car horn 2.wav,3.514,samples 1
8,car horn 3.wav,3.939,samples 1
9,car horn 1.wav,0.750,samples 1


In [81]:
def ComputeClassDetails(meta_data_df):
    
    """This function that will calculate the total number of files in the folder/class
        and also their total duration in HH:MM:SS format"""
    
    result_df = meta_data_df.groupby('Class').agg(
        {'Filename': 'count', 'Seconds': 'sum'}
    ).reset_index()
    result_df.rename(columns={'Filename': 'No. of Samples'}, inplace=True)
    result_df['Seconds'] = result_df['Seconds'].round().astype(int)
    result_df['Total Duration'] = pd.to_timedelta(result_df['Seconds'], unit='s').apply(lambda x: str(x).split()[-1])
    result_df = result_df.drop(columns=['Seconds'])
    return result_df


meta_data_df = pd.read_csv('/Users/rahulprem/Uni/SIT782/TestSet/species_metadata.csv')

ComputeClassDetails(meta_data_df)

,Class,No. of Samples,Total Duration
0,samples 1,3,00:00:08
1,samples 2,4,00:00:06
2,samples 3,3,00:00:33
